<a href="https://colab.research.google.com/github/ImenMasmoudiEm/ECOC-ALgorithm/blob/main/Ensembling_Methode_ECOC_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import os 
os.chdir('/content/drive/MyDrive/All/Projects/Ing Internship/Data')

#Reading Data


In [3]:
Tunnel=pd.read_excel("/content/drive/MyDrive/All/Projects/Ing Internship/Data/final-dataset.xlsx")

In [4]:
#For the Tunnal Dataset
Tunnel['classe']=Tunnel['classe'].replace("hate", int(2))
Tunnel['classe']=Tunnel['classe'].replace("normal", int(0)) 
Tunnel['classe']=Tunnel['classe'].replace("abusive", int(1))


TunnelS = Tunnel['commentaire']
TunnelL = Tunnel['classe']

TunnalL = [int(i) for i in TunnelL]

In [5]:
training_size=int(len(TunnelS)*0.9) 

training_sentences = TunnelS[0:training_size]
TunnelS = TunnelS[training_size:]
TunnelL = TunnelL[training_size:]

tokenizer = Tokenizer(num_words=3000, oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)

vocab_size = len(tokenizer.word_index) + 1 

word_index = tokenizer.word_index

#For Tunnel Dataset
TunnelS = tokenizer.texts_to_sequences(TunnelS)
TunnelS207 = pad_sequences(TunnelS, maxlen=207, padding='post', truncating='post')
TunnelS388 = pad_sequences(TunnelS, maxlen=388, padding='post', truncating='post')

In [6]:
TunnelS207[1]

array([ 307,   61,  307, 2079,  169,   14,   43, 2790, 1650, 2079, 2807,
        653,    1,    1,    3,  243,    1,    1,    1,    3,  212,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [7]:
#Importing the models
M1B09=tf.keras.models.load_model("/content/drive/MyDrive/All/Projects/Ing Internship/Data/Balanced 09/Model1Balanced09.h5")
M1U09=tf.keras.models.load_model("/content/drive/MyDrive/All/Projects/Ing Internship/Data/Unbalanced 09/Model1Unbalanced09.h5")
M2B09=tf.keras.models.load_model("/content/drive/MyDrive/All/Projects/Ing Internship/Data/Balanced 09/Model2Balanced09.h5")
M2U09=tf.keras.models.load_model("/content/drive/MyDrive/All/Projects/Ing Internship/Data/Unbalanced 09/Model2Unbalanced09.h5")
M3B09=tf.keras.models.load_model("/content/drive/MyDrive/All/Projects/Ing Internship/Data/Balanced 09/Model3Balanced09.h5")
M3U09=tf.keras.models.load_model("/content/drive/MyDrive/All/Projects/Ing Internship/Data/Unbalanced 09/Model3Unbalanced09.h5")

In [8]:
#Predictions for Tunnel
TunnelP11=M1B09.predict(TunnelS207)
TunnelP21=M1U09.predict(TunnelS388)
TunnelP31=M2B09.predict(TunnelS207)
TunnelP41=M2U09.predict(TunnelS388)
TunnelP51=M3B09.predict(TunnelS207)
TunnelP61=M3U09.predict(TunnelS388)

72/72 [==============================] - 14s 160ms/step


In [9]:
#First approach: Argmax and then look for the max

def fcount(c,L):
  r=0
  for i in L:
    if i==c:
      r+=1
  return r

def MaxV(L1,L2,L3,L4,L5,L6):
  L=np.zeros(len(L1))
  for i in range(len(L1)):
    if (fcount(2,[L1[i],L2[i],L3[i],L4[i],L5[i],L6[i]])>fcount(0,[L1[i],L2[i],L3[i],L4[i],L5[i],L6[i]])) and (fcount(2,[L1[i],L2[i],L3[i],L4[i],L5[i],L6[i]])>fcount(1,[L1[i],L2[i],L3[i],L4[i],L5[i],L6[i]])):
      L[i]=2
    elif (fcount(1,[L1[i],L2[i],L3[i],L4[i],L5[i],L6[i]])>fcount(0,[L1[i],L2[i],L3[i],L4[i],L5[i],L6[i]])):
      L[i]=1
    else:
      L[i]=0
  return L

#For the Tunnel Dataset
TunnelP1=np.argmax(TunnelP11, axis=1).astype(int)
TunnelP2=np.argmax(TunnelP21, axis=1).astype(int)
TunnelP3=np.argmax(TunnelP31, axis=1).astype(int)
TunnelP4=np.argmax(TunnelP41, axis=1).astype(int)
TunnelP5=np.argmax(TunnelP51, axis=1).astype(int)
TunnelP6=np.argmax(TunnelP61, axis=1).astype(int)

TunnelP=MaxV(TunnelP1,TunnelP2,TunnelP3,TunnelP4,TunnelP5,TunnelP6)

cm = confusion_matrix(TunnelL, TunnelP)
print('Testing on the Tunnel Dataset')
print(cm)
print(classification_report(TunnelL, TunnelP, labels=[0,1,2]))

Testing on the Tunnel Dataset
[[1256    3    6]
 [ 135  268   11]
 [ 187  100  338]]
              precision    recall  f1-score   support

           0       0.80      0.99      0.88      1265
           1       0.72      0.65      0.68       414
           2       0.95      0.54      0.69       625

    accuracy                           0.81      2304
   macro avg       0.82      0.73      0.75      2304
weighted avg       0.83      0.81      0.79      2304



In [10]:
#Second approach: We add the predictions and then we apply Argmax

#Testing on the Tunnel Dataset
TunnelPF=np.asarray(TunnelP11)+np.asarray(TunnelP21)+np.asarray(TunnelP31)+np.asarray(TunnelP41)+np.asarray(TunnelP51)+np.asarray(TunnelP61)
TunnelPF=np.argmax(TunnelPF, axis=1).astype(int)

cm = confusion_matrix(TunnelL, TunnelPF)
print('Testing on the Tunnel Dataset')
print(cm)
print(classification_report(TunnelL, TunnelPF, labels=[0,1,2]))

Testing on the Tunnel Dataset
[[1157   41   67]
 [  63  292   59]
 [  90   50  485]]
              precision    recall  f1-score   support

           0       0.88      0.91      0.90      1265
           1       0.76      0.71      0.73       414
           2       0.79      0.78      0.78       625

    accuracy                           0.84      2304
   macro avg       0.81      0.80      0.81      2304
weighted avg       0.84      0.84      0.84      2304



In [11]:
#Third approach: We use a Neural Network Model

#The Tunnel Dataset
TunnelL=np.asarray(TunnelL)
TunnelP=np.asarray(TunnelP)
XTunnel=[]
for i in range(len(TunnelL)):
  #print([TunnelP1[i],TunnelP2[i],TunnelP3[i],TunnelP4[i],TunnelP5[i],TunnelP6[i]]," the true value is: ", TunnelL[i], "P= ", TunnelP[i])
  XTunnel.append([TunnelP1[i],TunnelP2[i],TunnelP3[i],TunnelP4[i],TunnelP5[i],TunnelP6[i]])

In [12]:
TunnelL=np.asarray(TunnelL)

XTunnel=np.asarray(XTunnel)

In [13]:
FEModelTunnel = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(6,)),
    tf.keras.layers.Dense(128, activation='sigmoid'),
    tf.keras.layers.Dense(24, activation='selu'),
    tf.keras.layers.Dense(3, activation='softmax')
])
FEModelTunnel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 6)                 0         
                                                                 
 dense (Dense)               (None, 128)               896       
                                                                 
 dense_1 (Dense)             (None, 24)                3096      
                                                                 
 dense_2 (Dense)             (None, 3)                 75        
                                                                 
Total params: 4,067
Trainable params: 4,067
Non-trainable params: 0
_________________________________________________________________


In [14]:
FEModelTunnel.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
history = FEModelTunnel.fit(XTunnel, TunnelL, epochs=50, verbose=1)

Epoch 1/50
72/72 [==============================] - 1s 2ms/step - loss: 0.7883 - accuracy: 0.6753
Epoch 2/50
72/72 [==============================] - 0s 2ms/step - loss: 0.4328 - accuracy: 0.8802
Epoch 3/50
72/72 [==============================] - 0s 2ms/step - loss: 0.3418 - accuracy: 0.9501
Epoch 4/50
72/72 [==============================] - 0s 2ms/step - loss: 0.3126 - accuracy: 0.9588
Epoch 5/50
72/72 [==============================] - 0s 2ms/step - loss: 0.3026 - accuracy: 0.9588
Epoch 6/50
72/72 [==============================] - 0s 2ms/step - loss: 0.3045 - accuracy: 0.9588
Epoch 7/50
72/72 [==============================] - 0s 2ms/step - loss: 0.2936 - accuracy: 0.9588
Epoch 8/50
72/72 [==============================] - 0s 2ms/step - loss: 0.2877 - accuracy: 0.9588
Epoch 9/50
72/72 [==============================] - 0s 2ms/step - loss: 0.2854 - accuracy: 0.9588
Epoch 10/50
72/72 [==============================] - 0s 2ms/step - loss: 0.2796 - accuracy: 0.9588
Epoch 11/50
72/72 [

In [15]:
TunnelFP=FEModelTunnel.predict(XTunnel)

72/72 [==============================] - 0s 1ms/step


In [16]:
#Testing Third approach on the Tunnel Dataset
TunnelFP=np.argmax(TunnelFP, axis=1).astype(int)
cm = confusion_matrix(TunnelL, TunnelFP)
print('Testing on the Tunnel Dataset')
print(cm)
print(classification_report(TunnelL, TunnelFP, labels=[0,1,2]))

Testing on the Tunnel Dataset
[[1254    2    9]
 [  15  375   24]
 [  33   12  580]]
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1265
           1       0.96      0.91      0.93       414
           2       0.95      0.93      0.94       625

    accuracy                           0.96      2304
   macro avg       0.96      0.94      0.95      2304
weighted avg       0.96      0.96      0.96      2304

